# Título   Crear clase sample

## Autor: Jose Chelquer
## Fecha de última modificación: 20/11/2024
## Descripción:

Crear un sample del dataset con proporciones parametrizadas y meses seleccionados

## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [20]:
prueba=False                     # temporario. Volver a false



vm=True              # Se está trabajando en la vm o local?
experimento="_____"  # partiendo de la base
tipo_registros=experimento[0]            # '_'completo s sampleado
con_preprocesamiento = experimento[1]    # p con preprocesamiento  '_' sin preprocesamiento
tipo_featuring=experimento[2]            # _: 155 base  s: simple    c: completo
rf=experimento[3]                        # _: sin RF   r: con RF

dict_tipo_registros={'_': 'completo', 's': 'sampleado'}
dict_con_preprocesamiento={'_': 'sin preprocesamiento', 'p':'con preprocesamiento'}
dict_tipo_featuring={'_': 'base', 's':' simple', 'c': 'completo'}
dict_rf={'_': 'sin RF', 'f':' con RF'}

print (f'Experimento {experimento}')
print (dict_tipo_registros[tipo_registros])
print (dict_con_preprocesamiento[con_preprocesamiento])
print (dict_tipo_featuring[tipo_featuring])
print (dict_rf[rf])



Experimento _____
completo
sin preprocesamiento
base
sin RF


In [21]:

usar_gdrive=not(vm)  # en vm no se usa google drive

feature_a_usar = 'clase_ternaria'     #feature a la que hacen referencia los valores de % a seleccionar
#sample_pct={'BAJA+1':1, 'BAJA+2':1, 'CONTINUA':.05}   # % de cada clase a obtener en el sample
sample_pct={'BAJA+1':1, 'BAJA+2':1, 'CONTINUA':0.1}   # % de cada clase a obtener en el sample
#meses_a_seleccionar = [202101, 202102, 202103, 202104, 202105, 202106]       # Versión para seleccionar meses específicos
if prueba:
    meses_a_seleccionar = [202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108]       # Versión para seleccionar meses específicos
    #meses_a_seleccionar=[]                                # Versión para seleccionar todos los meses
    mes_test=202109         #el mes test va completo. SIEMPRE se lo selecciona
else:
    meses_a_seleccionar = [202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108]       # Versión para seleccionar meses específicos
    #meses_a_seleccionar=[]                                # Versión para seleccionar todos los meses
    mes_test=202109         #el mes test va completo. SIEMPRE se lo selecciona

In [22]:
mes_test

202109

## Input

< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

In [23]:
dataset_path = '~/buckets/b1/datasets' 
dataset_file=f"k3__{con_preprocesamiento}{tipo_featuring}{rf}.parquet.gz"
print (dataset_path)
print ("Se samplea {dataset_file)")


~/buckets/b1/datasets
Se samplea {dataset_file)


## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [24]:
# En la salida se marca que es sampleado
if prueba:
    output_file="k3_sample.parquet.gz"
else:
    output_file=f"k3_s{con_preprocesamiento}{tipo_featuring}{rf}.parquet.gz"
print (f'creando {output_file}')

creando k3_s___.parquet.gz


## Procesos

### Paquetes necesarios

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [25]:
import os
import pandas as pd

def cant_meses(df):
  return len(df['foto_mes'].unique())

In [26]:
if usar_gdrive:
    print ('Cargando google drive')
    from google.colab import drive
    drive.mount('/content/drive')

In [27]:
input_gz=dataset_file.endswith('.gz')
output_gz=output_file.endswith('.gz')

In [28]:
# leer el dataset
file_path = os.path.join(dataset_path, dataset_file)
print (f'leyendo {file_path}')
df = pd.read_parquet(file_path)

leyendo ~/buckets/b1/datasets/k3_____.parquet.gz


In [29]:
def contabilizar(df):
    print (f'Total de filas y features: \n{df.shape}')
    print (f'Filas por mes:\n {df['foto_mes'].value_counts()}')
    print (f'Casos por mes y clase:\n{pd.crosstab(df['foto_mes'], df['clase_ternaria'])}')
contabilizar(df)

Total de filas y features: 
(4901237, 155)
Filas por mes:
 foto_mes
202109    165644
202108    165442
202107    165152
202106    164876
202105    164623
202104    164090
202103    163685
202102    162646
202101    162026
202012    161526
202011    160742
202010    159731
202009    158371
202008    157058
202007    155764
202006    153757
202005    151261
202004    149872
202003    149356
202002    147109
202001    143966
201912    140661
201911    138667
201910    136682
201909    134314
201908    132664
201907    130724
201906    129186
201905    127659
201904    126996
201903    126436
201902    125799
201901    124752
Name: count, dtype: int64
Casos por mes y clase:
clase_ternaria  BAJA+1  BAJA+2  CONTINUA
foto_mes                                
201901             688     718    123346
201902             720     688    124391
201903             688     760    124988
201904             759     579    125658
201905             580     660    126419
201906             662     608    1

In [30]:
df_out=df

In [31]:
# Seleccionar meses
if meses_a_seleccionar:
    print (f'seleccionando meses {meses_a_seleccionar}')
    df_out=df_out[df_out['foto_mes'].isin(meses_a_seleccionar)]
    print (f'quedaron {df_out.shape[0]} filas')
contabilizar(df_out)

seleccionando meses [202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108]
quedaron 2265732 filas
Total de filas y features: 
(2265732, 155)
Filas por mes:
 foto_mes
202108    165442
202107    165152
202106    164876
202105    164623
202104    164090
202103    163685
202102    162646
202101    162026
202012    161526
202011    160742
202010    159731
202009    158371
202008    157058
202007    155764
Name: count, dtype: int64
Casos por mes y clase:
clase_ternaria  BAJA+1  BAJA+2  CONTINUA
foto_mes                                
202007             627     542    154595
202008             544     472    156042
202009             474     564    157333
202010             565     488    158678
202011             490     646    159606
202012             649     634    160243
202101             635     785    160606
202102             785    1017    160844
202103            1020     981    161684
202104             982    1189    1619

In [32]:
# muestrar
# Función para filtrar y muestrear por cada valor del feature
def filtrar_muestrear(df, feature, feat_grupos, sample_pct):
    grupos=df[feat_grupos].unique()
    print (grupos)
    df_sampled = pd.concat([
        df[(df[feature] == clase) & (df[feat_grupos]==grupo)].sample(frac=sample_pct[clase], random_state=42)
        for clase in sample_pct
        for grupo in grupos
    ])
    return df_sampled




# Aplicamos la función al DataFrame
df_out = filtrar_muestrear(df_out, feature_a_usar, 'foto_mes', sample_pct)

print (f'quedaron {df_out.shape[0]} filas')


[202007 202008 202009 202010 202011 202012 202101 202102 202103 202104
 202105 202106 202107 202108]
quedaron 393956 filas


In [33]:
contabilizar(df_out)

Total de filas y features: 
(393956, 155)
Filas por mes:
 foto_mes
202108    165442
202107     18647
202104     18363
202105     18352
202106     18271
202103     18169
202102     17886
202101     17481
202012     17307
202011     17097
202010     16921
202009     16771
202007     16629
202008     16620
Name: count, dtype: int64
Casos por mes y clase:
clase_ternaria  BAJA+1  BAJA+2  CONTINUA
foto_mes                                
202007             627     542     15460
202008             544     472     15604
202009             474     564     15733
202010             565     488     15868
202011             490     646     15961
202012             649     634     16024
202101             635     785     16061
202102             785    1017     16084
202103            1020     981     16168
202104             982    1189     16192
202105            1189     911     16252
202106             908    1074     16289
202107            1075    1294     16278
202108            1296  164146 

In [34]:
# Elimino el mes_test si por alguna razón quedó ahí
# Por las dudas, se eliminan registros de ese mes:
df_out = df_out[df_out['foto_mes'] != mes_test]



In [35]:
# Agrego completo el mes_test
# Filtra las filas de df que cumplen la condición
df_mes_test = df[df["foto_mes"] == mes_test]

# Agrega esas filas a df_out
df_out = pd.concat([df_out, df_mes_test], ignore_index=True)

In [36]:
contabilizar(df_out)

Total de filas y features: 
(559600, 155)
Filas por mes:
 foto_mes
202109    165644
202108    165442
202107     18647
202104     18363
202105     18352
202106     18271
202103     18169
202102     17886
202101     17481
202012     17307
202011     17097
202010     16921
202009     16771
202007     16629
202008     16620
Name: count, dtype: int64
Casos por mes y clase:
clase_ternaria  BAJA+1  BAJA+2  CONTINUA
foto_mes                                
202007             627     542     15460
202008             544     472     15604
202009             474     564     15733
202010             565     488     15868
202011             490     646     15961
202012             649     634     16024
202101             635     785     16061
202102             785    1017     16084
202103            1020     981     16168
202104             982    1189     16192
202105            1189     911     16252
202106             908    1074     16289
202107            1075    1294     16278
202108        

In [37]:
output_file_path=os.path.join(dataset_path, output_file)
if output_gz:
    df_out.to_parquet(output_file_path, index=False, compression='gzip')
else:
    df_out.to_parquet(output_file_path, index=False)
    

In [38]:
# Cantidad de variables
num_variables=len(df.columns)
# Filtra las columnas que empiezan con 'tree'
tree_columns = [col for col in df.columns if col.startswith('tree')]
# Cuenta el número de columnas
num_tree_columns = len(tree_columns)
print (f"Cantidad total de variables {num_variables}")
print(f"Cantidad de columnas que empiezan con 'tree': {num_tree_columns}")
print (f"Cantidad de variables normales: {num_variables-num_tree_columns}")
print (f"Archivo grabado: {output_file}")


Cantidad total de variables 155
Cantidad de columnas que empiezan con 'tree': 0
Cantidad de variables normales: 155
Archivo grabado: k3_s___.parquet.gz
